<a href="https://colab.research.google.com/github/yuriao/API/blob/master/commonlit_bert_090323.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I will explore how to use basic BERT network for scoring, based only on students' response

### References
-

In [ ]:
from google.colab import drive
drive.mount('commonLit_data')

Drive already mounted at commonLit_data; to attempt to forcibly remount, call drive.mount("commonLit_data", force_remount=True).


In [ ]:
!mkdir working

mkdir: cannot create directory ‘working’: File exists


In [ ]:
!pip install transformers[torch]

## Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline, AdamW
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
import logging
import warnings
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)

## Dataload

In [ ]:
DATA_DIR = "/content/commonLit_data/MyDrive/commonLit_data/"

#prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
#prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")


In [ ]:
summaries_train.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


## EDA: Check nans

In [ ]:
summaries_train.isna().sum()

student_id    0
prompt_id     0
text          0
content       0
wording       0
dtype: int64

In [ ]:
summaries_test.isna().sum()

student_id    0
prompt_id     0
text          0
dtype: int64

data is good

## Tokenizer and model

In [ ]:
#MODEL_DIR = "kaggle/input/huggingface-bert/"
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)

## encoding training text

In [ ]:
encodings = tokenizer(list(summaries_train['text']), return_tensors="pt", padding=True, truncation=True, max_length=32)
test_encodings = tokenizer(list(summaries_test['text']), return_tensors="pt", padding=True, truncation=True, max_length=32)

## Convert scores to tensors

In [ ]:
labels = torch.tensor(summaries_train[['content','wording']].values)

## to GPU

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Dataset class

In [ ]:
class RegressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

## fine-tuning metric

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## kFold training

In [ ]:
def train(config=None):

    # set training arguments
    training_args = TrainingArguments(
        output_dir='working',
	      #report_to='wandb',  # Turn on Weights & Biases logging
        num_train_epochs=5,
        learning_rate=config.learning_rate,
        weight_decay=config.weight_decay,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=16,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,
        remove_unused_columns=False,
        overwrite_output_dir=True,
        greater_is_better=False,
        fp16=True
    )


    # define training loop
    trainer = Trainer(
        # model,
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )


    # start training loop
    trainer.train()


In [ ]:
kfold=KFold(n_splits=2)

In [ ]:
#models=[]
for fold, (train_idx, val_idx) in enumerate(kfold.split(summaries_train['text'])):
    print(f"Fold {fold + 1}")

    train_encodings = {key: val[train_idx] for key, val in encodings.items()}
    val_encodings = {key: val[val_idx] for key, val in encodings.items()}
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]

    # Initialize datasets
    train_dataset = RegressionDataset(train_encodings, train_labels)
    val_dataset = RegressionDataset(val_encodings, val_labels)

    # Initialize the model for each fold
    #model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # num_labels=2 for dual regression

    wandb.agent(sweep_id, train, count=5)

Fold 1


wandb: Agent Starting Run: 6j9k5rpj with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 7.160544340661339e-05
wandb: 	weight_decay: 0.5


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Rmse
1,-0.870500,-2.342122,8.067143
2,-2.517200,-3.080933,11.002311
3,-3.315300,-3.398563,10.253956
4,-3.939000,-3.766501,11.738411
5,-4.698700,-4.047653,11.920138


eval/loss,█▅▄▂▁
eval/rmse,▁▆▅██
eval/runtime,▁▆█▆█
eval/samples_per_second,█▃▁▃▁
eval/steps_per_second,█▃▁▃▁
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
train/learning_rate,█▆▅▃▁
train/loss,█▅▄▂▁
train/total_flos,▁
train/train_loss,▁


...

Epoch,Training Loss,Validation Loss,Rmse
1,-5.999800,-8.260206,16.333075
2,-6.319100,-8.385247,16.891957
3,-6.583600,-8.451233,16.982429
4,-6.715900,-8.498399,17.043579
5,-6.818500,-8.488909,17.058944


Exception in thread Thread-36 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 441, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 668, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
RuntimeError: [enforce fail at inline_container.cc:471] . PytorchStreamWriter failed writing file data/97: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-19-7be1dc31d4e8>", line 40, in train
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1999, in _inner_training_loop
    self.control = self.callback_handler.on_train_end(args, self.state, self.control)
  F

wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Rmse
1,-6.722500,-8.501080,17.131475
2,-6.866600,-8.401447,17.296046
3,-6.891400,-8.491794,17.231647
4,-6.991400,-8.502128,17.252000
5,-7.005300,-8.513878,17.250081


Exception in thread Thread-39 (_run_job):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 441, in save
    _save(obj, opened_zipfile, pickle_module, pickle_protocol)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 668, in _save
    zip_file.write_record(name, storage.data_ptr(), num_bytes)
RuntimeError: [enforce fail at inline_container.cc:471] . PytorchStreamWriter failed writing file data/97: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-19-7be1dc31d4e8>", line 40, in train
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1555, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1999, in _inner_training_loop
    self.control = self.callback_handler.on_train_end(args, self.state, self.control)
  F

## Predict

In [ ]:
with torch.no_grad():
    test_ids = []
    test_attention_mask = []
    test_ids.append(test_encodings['input_ids'])
    test_attention_mask.append(test_encodings['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)
    predictions = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device)).logits

In [ ]:
print(predictions)

tensor([[-19.7500, -22.2500],
        [-19.7500, -22.2500],
        [-19.7500, -22.2500],
        [-19.7500, -22.2500]], device='cuda:0')


## Create Submission file

In [ ]:
summaries_test[["content", "wording"]]=predictions.cpu()
summaries_test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)